In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SCRAPI Prebuilt Development Notebook
In this notebook, we provide a launching point for developement on any Dialogflow CX Agent.

We will show you how to load all of the Agent Resources so that you can begin development on any area of your Agent.  
The motivation behind this notebook is to provide a checkpoint for you to start any DFCX development or scripting project.

## Prerequisites
- Ensure you have a GCP Service Account key with the following privileges assigned to it, based on your needs:
  - Dialogflow API Reader, use this permission level if you are just exploring the Dialogflow CX resources and developing offline
  - Dialogflow API Admin, use this permissions level if you are planning to Read/Write to your Dialogflow CX Agent and/or resources

In [3]:
# If you haven't already, make sure you install the `dfcx-scrapi` library

!pip install dfcx-scrapi --quiet

# Imports
During import, Colab will ask you to auth with your Google credentials.  
The creds are used to access Google Sheets where your training data lives.

In [4]:
import os
import time

import numpy as np
import pandas as pd

from dfcx_scrapi.core.agents import Agents
from dfcx_scrapi.core.flows import Flows
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.core.pages import Pages
from dfcx_scrapi.core.webhooks import Webhooks
from dfcx_scrapi.core.entity_types import EntityTypes
from dfcx_scrapi.core.environments import Environments
from dfcx_scrapi.core.transition_route_groups import TransitionRouteGroups
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions

from google.cloud.dialogflowcx_v3beta1 import types

## -- Uncomment the below section if you want to auth with Google Drive -- ##
# from google.colab import auth
# from google.auth import default

# # Mount your Google Drive.
# from google.colab import drive
# drive.mount("/content/drive")

# User Inputs
In the next section, we will collect runtime variables needed to execute this notebook.   
This should be the only cell of the notebook you need to edit in order for this notebook to run.

Getting an the training phrsaes data from your existing DFCX agent requires the following information:
- `creds_path`, path to your service account credentials file.
- `project_id`, your GCP project ID
- `agent_display_name`, the display name of your Dialogflow CX Agent
- `gcp_region`, (Optional) the GCP Region your Dialogflow CX Agent resides in
- `gcs_bucket_uri`, (Optional) a GCS bucket used to store data for your Dialogflow CX Agent

In [11]:
creds_path = "<YOUR_CREDS_PATH_HERE>"
project_id = "<YOUR_GCP_PROJECT_ID>"
agent_display_name = "<YOUR_AGENT_DISPLAY_NAME>"
gcp_region = "<GCP_REGION>" # Optional
gcs_bucket_uri = "<GCS_BUCKET_URI>" # Optional

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = creds_path

# Load Agent
First, we will instantiate our core classes.

In [12]:
# Instantiate the core classes
agents_instance = Agents(creds_path=creds_path)
flows_instance = Flows(creds_path=creds_path)
intents_instance = Intents(creds_path=creds_path)
pages_instance = Pages(creds_path=creds_path)
entity_types_instance = EntityTypes(creds_path=creds_path)
webhooks_instance = Webhooks(creds_path=creds_path)
environments_instance = Environments(creds_path=creds_path)
route_groups_instance = TransitionRouteGroups(creds_path=creds_path)

# Instantiate the tools classes
dffx_instance = DataframeFunctions(creds_path=creds_path)

# Get the Agent and its ID
my_agent = agents_instance.get_agent_by_display_name(
    project_id=project_id, display_name=agent_display_name
)

agent_id = my_agent.name

2022-09-29 15:01:46 INFO     create dfcx creds /Users/pmarlow/eng/creds/nj-pods-dev-pmarlow.json


# Load Agent Components
In this section, we'll load up all of the various Agent Resource components like Intents, Flows, Pages, etc.

## Load Flows and Pages

In [15]:
# Load the flows
all_flows = flows_instance.list_flows(agent_id=agent_id)

flows_map = flows_instance.get_flows_map(agent_id=agent_id)
flows_map_reverse = flows_instance.get_flows_map(
    agent_id=agent_id, reverse=True
)

# Load the pages
all_pages_per_flow = {}

pages_map_per_flow = {}
pages_map_per_flow_reverse = {}

for flow_name, flow_id in flows_map_reverse.items():
    all_pages_per_flow[flow_name] = pages_instance.list_pages(flow_id=flow_id)
    time.sleep(1)
    
    pages_map_per_flow[flow_name] = pages_instance.get_pages_map(
        flow_id=flow_id
    )
    time.sleep(1)
    pages_map_per_flow_reverse[flow_name] = pages_instance.get_pages_map(
        flow_id=flow_id, reverse=True
    )

## Load Intents and Entity Types

In [16]:
# Load the intents
all_intents = intents_instance.list_intents(agent_id=agent_id)

intents_map = intents_instance.get_intents_map(agent_id=agent_id)
intents_map_reverse = intents_instance.get_intents_map(
    agent_id=agent_id, reverse=True
)

# Load the entity types
all_entity_types = entity_types_instance.list_entity_types(agent_id=agent_id)

entity_types_map = entity_types_instance.get_entities_map(agent_id=agent_id)
entity_types_map_reverse = entity_types_instance.get_entities_map(
    agent_id=agent_id, reverse=True
)

## Load Webhooks, Environments, and Transition Route Groups

In [17]:
# Load the webhooks
all_webhooks = webhooks_instance.list_webhooks(agent_id=agent_id)

webhooks_map = webhooks_instance.get_webhooks_map(agent_id=agent_id)
webhooks_map_reverse = webhooks_instance.get_webhooks_map(
    agent_id=agent_id, reverse=True
)

# Load the environments
all_environments = environments_instance.list_environments(agent_id=agent_id)

environments_map = environments_instance.get_environments_map(agent_id=agent_id)
environments_map_reverse = environments_instance.get_environments_map(
    agent_id=agent_id, reverse=True
)

# Load the transition route groups
all_route_groups_per_flow = {}

route_groups_map_per_flow = {}
route_groups_map_per_flow_reverse = {}

for flow_name, flow_id in flows_map_reverse.items():
    route_groups_list = route_groups_instance.list_transition_route_groups(
        flow_id=flow_id
    )
    all_route_groups_per_flow[flow_name] = route_groups_list
    time.sleep(1)
    
    route_groups_map = route_groups_instance.get_route_groups_map(
        flow_id=flow_id
    )
    route_groups_map_per_flow[flow_name] = route_groups_map
    time.sleep(1)
    route_groups_map_reverse = route_groups_instance.get_route_groups_map(
        flow_id=flow_id, reverse=True
    )
    route_groups_map_per_flow_reverse[flow_name] = route_groups_map_reverse

# Start Coding!
Now that we have all of our Agent Resources loaded, you can start exploring them in many ways below.  

In [18]:
# < - - Amazing Code Goes Here! -- > 

# Final Thoughts and Wrap-Up
In this notebook, we've provided a quick start template to load all of your Dialogflow CX Agent Resources so you can start exploring, scripting, and building prodution level bots!

Feel free to import more classes and try them too!  
We suggest checking out the `tools/` directory for other great things you can do with SCRAPI!

Happy Coding!